In [3]:
import os
import sys
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from efficientNetV2Model import EfficientNetV2SWheatModel
from dataLoaderFunc import loadSplitData, createLoader

c:\Users\pacha\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            outputs = model(rgb_batch, dsm_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 20 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Compute Validation Loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                outputs = model(rgb_batch, dsm_batch)
                loss = criterion(outputs, label_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save Best Model
        torch.save(model.state_dict(), "efficientNetV2_wheat_model.pth")


In [5]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [6]:

# ✅ Initialize Model, Loss Function, and Optimizer

# ✅ Universal device selection (works on both Mac M2 & Windows with RTX 4060)
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

model = EfficientNetV2SWheatModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate for EfficientNet
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
# ✅ Start Training

✅ Using device: cuda


In [7]:
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10 | Batch 0/2990 | Loss: 112145.0703
Epoch 1/10 | Batch 20/2990 | Loss: 42102.0312
Epoch 1/10 | Batch 40/2990 | Loss: 9607.5234
Epoch 1/10 | Batch 60/2990 | Loss: 5663.7197
Epoch 1/10 | Batch 80/2990 | Loss: 17251.5469
Epoch 1/10 | Batch 100/2990 | Loss: 6245.5566
Epoch 1/10 | Batch 120/2990 | Loss: 5277.8018
Epoch 1/10 | Batch 140/2990 | Loss: 8148.7012
Epoch 1/10 | Batch 160/2990 | Loss: 9858.2168
Epoch 1/10 | Batch 180/2990 | Loss: 5364.1572
Epoch 1/10 | Batch 200/2990 | Loss: 8010.6533
Epoch 1/10 | Batch 220/2990 | Loss: 3866.4529
Epoch 1/10 | Batch 240/2990 | Loss: 3886.4302
Epoch 1/10 | Batch 260/2990 | Loss: 2637.2451
Epoch 1/10 | Batch 280/2990 | Loss: 6688.7969
Epoch 1/10 | Batch 300/2990 | Loss: 2031.9623
Epoch 1/10 | Batch 320/2990 | Loss: 3313.1519
Epoch 1/10 | Batch 340/2990 | Loss: 3343.2329
Epoch 1/10 | Batch 360/2990 | Loss: 4436.6436
Epoch 1/10 | Batch 380/2990 | Loss: 6868.3984
Epoch 1/10 | Batch 400/2990 | Loss: 4142.9385
Epoch 1/10 | Batch 420/2990 | Loss: 